# Setup

In [1]:
%matplotlib notebook
from matplotlib import pyplot as plt
plt.style.use('seaborn-darkgrid')

LOG_PLOT = False
NDAYS=100
from matplotlib import cm
import os
from helper import *
from datetime import datetime
from ecdc import ECDC


## Initialisation

In [2]:
ecdc = ECDC()

In [3]:
 ecdc.get_cases_by_country('Netherlands')

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
2925,2021-03-01,1,3,2021,1087303,15544,Netherlands,NL,NLD,17407585,Europe
2924,2021-03-02,2,3,2021,3753,21,Netherlands,NL,NLD,17407585,Europe
2923,2021-03-03,3,3,2021,3976,65,Netherlands,NL,NLD,17407585,Europe
2922,2021-03-04,4,3,2021,4991,48,Netherlands,NL,NLD,17407585,Europe
2921,2021-03-05,5,3,2021,4110,33,Netherlands,NL,NLD,17407585,Europe
...,...,...,...,...,...,...,...,...,...,...,...
2797,2021-07-07,7,7,2021,2204,1,Netherlands,NL,NLD,17407585,Europe
2796,2021-07-08,8,7,2021,3642,2,Netherlands,NL,NLD,17407585,Europe
2795,2021-07-09,9,7,2021,5422,2,Netherlands,NL,NLD,17407585,Europe
2794,2021-07-10,10,7,2021,6923,2,Netherlands,NL,NLD,17407585,Europe


In [4]:
ecdc.cases

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2020,continentExp
3856,2021-02-28,28,2,2021,3109689,69142,Spain,ES,ESP,47332614,Europe
1994,2021-03-01,1,3,2021,219592,4319,Ireland,IE,IRL,4964440,Europe
398,2021-03-01,1,3,2021,247038,10191,Bulgaria,BG,BGR,6951482,Europe
531,2021-03-01,1,3,2021,242973,5526,Croatia,HR,HRV,4058165,Europe
664,2021-03-01,1,3,2021,34707,231,Cyprus,CY,CYP,888005,Europe
...,...,...,...,...,...,...,...,...,...,...,...
1330,2021-07-11,11,7,2021,745,6,Germany,DE,DEU,83166711,Europe
1463,2021-07-11,11,7,2021,2327,2,Greece,EL,GRC,10718565,Europe
2926,2021-07-11,11,7,2021,0,0,Norway,NO,NOR,5367580,Europe
1729,2021-07-11,11,7,2021,0,0,Iceland,IS,ISL,364134,Europe
